In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
training_data= datasets.FashionMNIST(
    root="data",
    train = True,
    download=True,
    transform=ToTensor()
)
test_data= datasets.FashionMNIST(
    root="data",
    train = False,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [17]:
img, label = training_data[0]
print(img.shape, type(label))

torch.Size([1, 28, 28]) <class 'int'>


In [18]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

imgs, labels = next(iter(train_dataloader))
print(labels.shape, imgs.shape)

torch.Size([64]) torch.Size([64, 1, 28, 28])


In [25]:
test = imgs[0]
test.shape
torch.flatten(test, start_dim=1).shape

torch.Size([1, 784])

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=512, bias=True),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)        

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /=size
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg_loss: {test_loss:>8f}\n")

In [32]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
----------------------------------------
loss: 2.165361 [   64/60000]
loss: 2.152876 [ 6464/60000]
loss: 2.099039 [12864/60000]
loss: 2.115060 [19264/60000]
loss: 2.081282 [25664/60000]
loss: 2.010416 [32064/60000]
loss: 2.046609 [38464/60000]
loss: 1.974996 [44864/60000]
loss: 1.966085 [51264/60000]
loss: 1.900592 [57664/60000]
Test Error:
 Accuracy: 58.2%, Avg_loss: 1.896716

Epoch 2
----------------------------------------
loss: 1.926005 [   64/60000]
loss: 1.894427 [ 6464/60000]
loss: 1.786571 [12864/60000]
loss: 1.826489 [19264/60000]
loss: 1.719592 [25664/60000]
loss: 1.673583 [32064/60000]
loss: 1.697004 [38464/60000]
loss: 1.606403 [44864/60000]
loss: 1.617171 [51264/60000]
loss: 1.513482 [57664/60000]
Test Error:
 Accuracy: 63.0%, Avg_loss: 1.525025

Epoch 3
----------------------------------------
loss: 1.590227 [   64/60000]
loss: 1.549032 [ 6464/60000]
loss: 1.409833 [12864/60000]
loss: 1.477485 [19264/60000]
loss: 1.351614 [25664/60000]
loss: 1.359314 [32064/60000]